In [1]:
%load_ext autoreload
%autoreload 

In [2]:

import os 
import shutil
import numpy as n
from datetime import date
from matplotlib import pyplot as plt
import time

os.chdir('/home/ali/packages/s2p-lbm/')
from scipy import stats
import suite2p
from suite3d import plot_utils as plot

In [3]:
from importlib import reload
from pathlib import Path

In [6]:
from suite3d.job import Job

job = Job('/mnt/md0/runs','SS003_2024-08-06_2', create=False)

   Found and loaded params from /mnt/md0/runs/s3d-SS003_2024-08-06_2/params.npy


In [7]:
timing = {'extraction': 96, 'segmentation': 3*60 + 18, 'corrmap' : 104, 'data_io' :}

SyntaxError: invalid syntax (3463373553.py, line 1)

In [10]:
# %%time
# # SAVE DATA FOR s2p Run
# from suite3d.io.s3dio import s3dio
# from suite3d import reference_image as ref
# import tifffile

# jobio = s3dio(job)

# raw_data_path = '/mnt/md0/runs/s2p-SS003_2024-08-06_2/raw'

# os.makedirs(raw_data_path,exist_ok=True)

# planes = n.array([6,7,8,9,10,11,12])
# fuse_shift = 7
# ys, xs = jobio._load_roi_start_pix_lbm(job.params)
# for tif in job.tifs:
#     print(tif)
#     data = jobio.load_data([tif], planes = job.params['planes'])
#     out_fname = tif.split('/')[-1].split('.')[0] + f'_{len(planes)}planes.tif'
#     out_path = os.path.join(raw_data_path, out_fname)
#     data = ref.fuse_mov(data, fuse_shift, n.sort(xs))[0]
#     data = n.swapaxes(data,0,1)        
#     tifffile.imsave(out_path, data)

[{'uid': '9A1B7D0F2A67F484', 'center': array([0, 0]), 'sizeXY': array([ 2.7  , 16.875]), 'pixXY': array([144, 900])}, {'uid': 'D13AC8ABC660883F', 'center': array([2.7, 0. ]), 'sizeXY': array([ 2.7  , 16.875]), 'pixXY': array([144, 900])}, {'uid': '66DCE42C83645264', 'center': array([5.4, 0. ]), 'sizeXY': array([ 2.7  , 16.875]), 'pixXY': array([144, 900])}, {'uid': '18B13E686766C950', 'center': array([0, 0]), 'sizeXY': array([ 2.7  , 16.875]), 'pixXY': array([144, 900])}, {'uid': '305F5B1ABEA43014', 'center': array([-2.7,  0. ]), 'sizeXY': array([ 2.7  , 16.875]), 'pixXY': array([144, 900])}, {'uid': '886741FECFC3BCA6', 'center': array([-5.4,  0. ]), 'sizeXY': array([ 2.7  , 16.875]), 'pixXY': array([144, 900])}]
[288 432 576 288 144   0]
/mnt/zortex-subjects/SS003/2024-08-06/2/2024-08-06_2_SS003_2P_00001_00001.tif
      Loading /mnt/zortex-subjects/SS003/2024-08-06/2/2024-08-06_2_SS003_2P_00001_00001.tif
    Loaded file into shared memory in 4.77 sec
    Workers completed in 0.85 sec


In [8]:
raw_data_path ='/mnt/md0/runs/s2p-SS003_2024-08-06_2/raw'

save_path =  Path('/mnt/md0/runs/s2p-SS003_2024-08-06_2/')

ops = suite2p.default_ops()
ops['look_one_level_down'] = True
ops['subfolders'] = []
ops['nplanes'] = 7
ops['fs'] = 4.12
ops['batch_size'] = 400
ops['threshold_scaling'] = 0.1
ops['tau'] = 1.3
ops['nonrigid'] = False
ops['ignore_flyback'] = []
ops['data_path'] = [raw_data_path]
ops['save_path0'] = str(save_path)
# print(ops)
ops['do_registration'] = 2

In [9]:
# %%time
# output_ops = suite2p.run_s2p(ops=ops)

In [10]:
plane_times = {'init' : 11, 'mcorr' : 28 + 29, 'segmentation' : 33, 'extraction' : 10}
plane_total = sum([plane_times[k] for k in plane_times])

In [11]:
total_time = 12*60 + 56
timings = {k : total_time * plane_times[k] / plane_total for k in plane_times.keys()}

In [12]:
timings_3d = {'extraction': 96, 'segmentation': 3*60 + 18, 'corrmap' : 104, 'init' : 127, 'mcorr' : 7*60+54 }

In [13]:
total_time_3d = sum(timings_3d[k] for k in timings_3d.keys())

In [31]:
all_ops = []
all_stats = []
Fs = []
spks = []
iscell = []
for plane in n.arange(ops['nplanes']):
    planedir = (save_path / 'suite2p' / f'plane{plane}')
    statx = planedir / 'stat.npy'
    opx = planedir / 'ops.npy'
    Fsx = planedir / 'F.npy'
    spksx = planedir / 'spks.npy'
    iscellx = planedir / 'iscell.npy'

    plane_stats = n.load(statx, allow_pickle=True)
    for i in range(len(plane_stats)):
        plane_stats[i]['plane_idx'] = int(plane)
    all_stats.append(plane_stats)
    all_ops.append(n.load(opx, allow_pickle=True).item())
    Fs.append(n.load(Fsx))
    spks.append(n.load(spksx))
    iscell.append(n.load(iscellx))

iscell = n.concatenate(iscell).astype(bool)

mean_img = n.stack([op['meanImg'] for op in all_ops])

ref_img = n.stack([op['refImg'] for op in all_ops])


# corr_map = n.stack([op['Vcorr'] for op in all_ops])

nz = mean_img.shape[0]

In [32]:
Fs = n.concatenate(Fs)[iscell[:,0]]
spks = n.concatenate(spks)[iscell[:,0]]

In [33]:
out_stats = [s for i,s in enumerate(n.concatenate(all_stats)) if iscell[i,0]]

In [35]:
n.save(save_path / 'F.npy', Fs)
n.save(save_path / 'spks.npy', spks)

In [44]:
meds = n.array([n.concatenate([(s['plane_idx'],), s['med']]) for s in out_stats])

In [46]:
n.save(save_path / 'meds.npy',meds)

In [37]:
n.save(save_path / 'stats.npy',out_stats)